In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("ENEMTeste").getOrCreate()

In [6]:
file_path = "enem2019/DADOS/MICRODADOS_ENEM_2019.csv"

In [7]:
# Leitura dos dados com Spark
enem = (
    spark
    .read
    .format("csv")
    .options(header=True, delimiter=";", encoding="latin1", inferSchema=True)
    .load(file_path)
)

In [8]:
enem.count()

5095270

## Calcular a média de idade e a média da nota em matemática por sexo e estado de RJ e SP

In [9]:
from pyspark.sql.functions import count, col, lit, mean

In [10]:
# Utilizando a API Dataframe
consulta = (
    enem
    .where(col("SG_UF_RESIDENCIA").isin(["RJ", "SP"]))
    .groupBy("TP_SEXO", "SG_UF_RESIDENCIA")
    .agg(
        mean(col("NU_IDADE")).alias("med_idade"),
        mean(col("NU_NOTA_MT")).alias("med_nota_mt")
    )
)

In [12]:
consulta.show(n=10, truncate=False)

+-------+----------------+------------------+-----------------+
|TP_SEXO|SG_UF_RESIDENCIA|med_idade         |med_nota_mt      |
+-------+----------------+------------------+-----------------+
|M      |SP              |20.975866306954437|578.2618169116553|
|F      |SP              |20.85638000983014 |532.5990530953453|
|F      |RJ              |22.613914048378653|520.3744438133432|
|M      |RJ              |22.750757959492297|567.9497019079913|
+-------+----------------+------------------+-----------------+



## Mesmos cálculos utilizando SQL

In [13]:
# Cria uma view da tabela
enem.createOrReplaceTempView("enem")

In [14]:
# Calcula a média de idade e a média da nota em matemática por sexo e estado Brasileiro para alunos que residem no estado de RJ e SP
consulta2 = spark.sql("""

    SELECT TP_SEXO, SG_UF_RESIDENCIA, AVG(NU_IDADE) as med_idade, AVG(NU_NOTA_MT) as med_nota_mt
    FROM enem
    WHERE SG_UF_RESIDENCIA IN ('RJ', 'SP')
    GROUP BY TP_SEXO, SG_UF_RESIDENCIA

""")

In [15]:
consulta2.show(n=10, truncate=False)

+-------+----------------+------------------+-----------------+
|TP_SEXO|SG_UF_RESIDENCIA|med_idade         |med_nota_mt      |
+-------+----------------+------------------+-----------------+
|M      |SP              |20.975866306954437|578.2618169116553|
|F      |SP              |20.85638000983014 |532.5990530953453|
|F      |RJ              |22.613914048378653|520.3744438133432|
|M      |RJ              |22.750757959492297|567.9497019079913|
+-------+----------------+------------------+-----------------+



## Calcula a média de idade, média da nota em matemática, média da nota em CH, desvio padrão da nota em MT, contagem de linhas por grupos de sexo, Estado e Municipio

In [17]:
[16]
consulta3 = spark.sql("""

    SELECT 
        tp_sexo, sg_uf_residencia, no_municipio_residencia, 
        avg(nu_idade) as med_idade, avg(nu_nota_mt) as med_nota_mt,
        stddev(nu_nota_mt) as std_nota_mt, count(1) as contagem
    FROM enem
    GROUP BY tp_sexo, sg_uf_residencia, no_municipio_residencia
    ORDER BY no_municipio_residencia, tp_sexo, sg_uf_residencia

""")

In [18]:
consulta3.show(100, truncate=False)

+-------+----------------+-----------------------+------------------+------------------+------------------+--------+
|tp_sexo|sg_uf_residencia|no_municipio_residencia|med_idade         |med_nota_mt       |std_nota_mt       |contagem|
+-------+----------------+-----------------------+------------------+------------------+------------------+--------+
|F      |GO              |Abadia de Goiás        |21.888297872340427|488.4504          |87.53847875040715 |188     |
|M      |GO              |Abadia de Goiás        |21.652631578947368|523.3237288135593 |100.18094549877554|95      |
|F      |MG              |Abadia dos Dourados    |19.746987951807228|506.6999999999999 |88.47680567729118 |83      |
|M      |MG              |Abadia dos Dourados    |20.210526315789473|541.9499999999999 |117.52050339387047|57      |
|F      |GO              |Abadiânia              |18.58282208588957 |502.8083333333332 |93.82494097171633 |163     |
|M      |GO              |Abadiânia              |19.71794871794